In [1]:
%%html
<style>
table th { text-align: left!important; }
table td { text-align: left!important; }
</style>

# 第6章 モデル（Model）

## 6.1 概要
Djangoは、「Django ORM」と呼ばれるORマッパーの機能を提供している  
Djangoには、マイグレーション機能が備わっている  
$\therefore$ Djangoを使った開発では、「モデル作成」→「テーブル作成」を繰り返して進めていくのが基本

## 6.2 モデルクラスの書き方
モデルクラスは、アプリケーションディレクトリ内の`models.py`ファイルに記述していく  
モデルクラスは、`django.db.models.Model`クラスを継承する必要がある  
通常、1つのモデルクラスは1つのテーブルに対応する  

### Bookクラスの実装例
~~~ python
from django.db import models

class Book(models.Model):
    """本モデル"""
    class Meta:
        # テーブル名を定義
        db_table = 'book'
        
    # テーブルのカラムに対応するフィールドを定義
    title = models.CharField(verbose_name = 'タイトル', max_length = 255)
    price = models.IntegerField(verbose_name = '価格', null = True, blank = True)
    
    def __str__(self):
        return self.title
~~~

### `Meta`クラス
モデル内部の`Meta`クラスの属性に、以下のようなモデルクラス全体に対する付加情報を設定する
- 対応するテーブル名  
  設定しない場合は、「<アプリケーション名>\_<モデルのクラス名をスネークケースにした文字列>」で自動的に名付けられる
- 複数カラムに対するインデックス
- ユニーク制約

### テーブルのカラム
カラムはモデルクラスのクラス属性として定義する  
クラス属性には`django.db.models.fields.Field`のサブクラスを利用する  

#### 主なFieldクラスとデータベースのカラムの型の対応

| Fieldクラス | カラムの型（SQLite） | カラムの型（MySQL） |
| :---------- | :------------------- | :------------------ |
| django.db.models.fields.BooleanField | bool | tinyint(1) |
| django.db.models.fields.CharField | varchar | varchar |
| django.db.models.fields.TextField | text | longtext |
| django.db.models.fields.IntegerField | integer | int(11) |
| django.db.models.fields.FloatField | real | double |
| django.db.models.fields.DateField | date | date |
| django.db.models.fields.DateTimeField | datetime | datetime(6) |

#### フィールドオプション
Fieldクラスに指定できる属性で、以下のようなものを定義出来る
- データベース上の制約
- `ModelForm`を利用したときの画面上の表示名やバリデーション

#### Firldクラスに指定できるフィールドオプション
| フィールドオプション | 説明 |
| :------------------- | :--- |
| verbose_name         | フィールド名 |
| null                 | データベースのNULL制約（デフォルト値は`False`：許可しない） |
| unique               | データベースのユニーク制約 |
| db_index             | データベースのインデックスを設定するかどうか |
| default              | レコード登録時に値が指定されなかったときのデフォルト値 |
| blank                | フォーム利用時に入力必須にするかどうか（デフォルト値は`False`） |
| max_length           | 文字列の最大文字数。`CharField`では指定必須 |
| choices              | フォーム利用時にセレクトボックスに表示する選択肢 |
| validators           | 文字種チェック等のバリデーションを指定 |
| error\_messages      | バリデーションNGの場合のエラーメッセージ |

### モデルクラスを書くときのその他の注意点
- 主キー（id）は自動で生成されるため、明示的に定義する必要なし
- 主キー（id）はレコードが登録されるたびに自動採番（1からインクリメント）
- 複合キーを主キーに出来ない（`Meta.unique_together`で代替するなど）
- 他テーブルへの一対一、多対一、多対多等のリレーションも定義可能（[次節](#6.3-「一対一」「多対一」「多対多」リレーションはどう定義するか？)を参照）
- \_\_str\_\_メソッドで管理サイトなどに表示される文字列を定義

## 6.3 「一対一」「多対一」「多対多」リレーションはどう定義するか？
Djangoで「一対一」「多対一」「多対多」のリレーションを定義するためには、以下の定義を使用する
- `OneToOneField`
- `FpreignKey`
- `ManyToManyField`

### 6.3.1 「一対一」のリレーション
#### 「一対一」の関係
ある種類のデータに対して他のデータが必ずひとつ（あるいはゼロ）しか関連付かないという関係性
例：
- 本と本の在庫情報
- システム利用者と（アカウント登録時に発行する）アクティベーションキー情報
- システム利用者とショッピングカート

#### 「一対一」のリレーションの実装例
~~~python
from django.db import models

class Book(models.Model):
    """本モデル"""
    title = models.CharField(verbose_name = 'タイトル', max_length = 255)
    
class BookStopck(models.Model):
    """本の在庫モデル"""
    book = models.OneToOneField(Nook, verbose_name = '本', on_delete = models.CASCADE)
    quantity = models.IntegerField(verbose_name = '在庫数', default = 0)
~~~

### 「多対一」のリレーション
#### 「多対一」の関係
「多対一」か「多対多」かを分類するには、一方から見たときに相手を「*同時に2つ以上*」紐付けることが出来るかどうかで相手が「多」で有るかどうかが決まる
例：
- 「注文」（多）と「システム利用者」（一）
- 「注文明細」（多）と「注文」（一）
- 「ブログ記事」（多）と「ブログの投稿者」（一）

Djangoでは「多」側のモデルに`django.db.models.ForeignKey`のフィールドを持たせることで実現出来る

#### 「多対一」のリレーションの実装例
~~~python
from django.db import models

class Publisher(models.Model):
    """出版社モデル"""
    name = models.CharField(verbose_name = '出版社名', max_length = 255)
    
class Book(models.Model):
    """本モデル"""
    ・・・
    publisher = models.ForeignKey(Publisher, verbose_name = '出版社', on_delete = models.CASCADE)
~~~

#### `on_delete`フィールド
参照先のオブジェクトが削除さえr多彩に自身のオブジェクトがどのような挙動をするかの設定  
`ForeignKey`と`OneToOneField`には`on_delete`属性をつけるようにする（Django2系では設定必須）

| 設定値 | 説明 |
| :----- | :--- |
| django.db.models.CASCADE | 自身のレコードも削除される |
| django.db.models.PROTECT | 自身のレコードは削除されない |
| django.db.models.SET_NULL | このフィールドにNULLがセットされる。<br />ただし、`null`オプションが`True`の場合のみ |
| django.db.models.SET_DEFAULT | このフィールドにデフォルト値がセットされる。<br />ただし、`default`オプションが設定されている場合のみ |


### 6.3.3 「多対多」のリレーション
#### 「多対多」の関係
どちらから見ても、同時に2つ以上紐付けることの出来る関係性  
実際のテーブル設計では、中間テーブルを作ることで実現するのが一般的  
Djangoでも実際は中間テーブルを作成するが、実装上は`django.db.models.fields.related.ManyToManyField`を付けるだけで対応可能

#### 「多対多」のリレーションの実装例
~~~python
from django.db import models

class Author(models.Model):
    """著者モデル"""
    name = models.CharField(verbose_name = '著者名', max_length = 255)
    
class Book(models.Model):
    """本モデル"""
    ・・・
    authors = models.ManyToManyField(Author, verbose_name = '著者')
~~~

## 6.4 よく使われる`User`モデル
### `User`モデル
Djangoがデフォルトで提供するクラス  
システム利用者（ユーザー）として利用するのが一般的

## 6.5 モデルマネージャとクエリセット
### モデルマネージャ
モデルは「モデルマネージャ」と呼ばれるデータベースのテーブルレベルのクエリ操作を提供するインターフェースを持つ  
モデルマネージャは直接データベースとはクエリのやり取りは行わない  
通常、モデルクラスに`object`という名前で保管されている

### クエリセットAPI
データベースと直接クエリをやり取りする

### モデルからテーブルを検索する
1. モデルのモデルマネージャを経由してクエリセットAPIのメソッドを呼び出す
1. クエリセットAPIが以下のどちらかを行う
   - データベースにアクセスしてモデルのオブジェクトやその他の結果を返す
   - データベースにはアクセスせずにクエリセットオブジェクトを返す
1. クエリセットオブジェクトを受け取った場合は引き続きクエリセットAPIを利用することが出来る

## 6.6 単体のオブジェクトを取得する
### `get`メソッド
データベースにアクセスし、モデルのオブジェクトを1件だけ取得する  
メソッドのキーワード引数としてモデルクラスのフィールド名に任意の値を指定することで検索条件を指定することが出来る  
また、`pk`（`id`でもOK）というキーワード引数を使うと、モデルに自動的に追加された主キーで検索する事ができる  
1件も見つからない場合はモデルクラスの`DoesNotExist`例外が発生する

#### `User`モデルで`username`が`admin`のオブジェクトを取得する
~~~python
User.objects.get(username='admin')
~~~

#### `User`モデルで`pk`が`2`のオブジェクトを取得する
~~~python
User.objects.get(pk=2)
~~~

## 6.7 複数のオブジェクトを取得する
複数のオブジェクトを取得する場合は、モデルマネージャを経由してクエリセットAPIの`all`または`filter`メソッドを利用する

### 6.7.1 `all()`メソッド
レコード全件に対応するオブジェクトのリストを取得する  
即座にデータベースにはアクセスせず、クエリセットオブジェクトを返す（遅延評価）  
結果が0件の場合は空のリストが返る  

#### `User`モデルのオブジェクトを全て取得する
~~~python
User.objects.all()
~~~

### 6.7.2 `filter()`メソッド
検索条件をつけてオブジェクトのリストを取得する  
メソッドのキーワード引数としてモデルクラスのフィールド名と任意の値を指定することで条件検索をすることが出来る  
`all()`と同様にクエリセットオブジェクトが返る

#### `User`モデルの`is_active`カラムが`True`のオブジェクトを検索する
~~~python
User.object.filter(is_active=True)
~~~

### 6.7.3 AND条件
#### AND条件を使った検索
タイトルが「Django Book」かつ、価格が1000円の本を検索する
~~~python
Book.objects.filter(title='Django Book', price=1000)
~~~

### 6.7.4 OR条件
`django.db.models.Q`と`|`を使う
#### OR条件を使った検索
タイトルが「Django Book」または価格が1000円の本を検索する
~~~python
from django.db.models import Q
Book.objects.filter(Q(title='Django Book') | Q(price=1000))
~~~

### 6.7.5 不等号（$>$, $<$, $>=$, $<=$）/IN句/LIKE句を使った検索
不等号やIN句、LIKE句を使った検索をする場合には以下の書式でキーワードを記述する
`<フィールド名>\_\_<不等号/IN句/LIKE句のキーワード>`

#### $>$を使った検索
`gt`（Greater Than）を使う  
例：価格が1000円より高いBookモデルのオブジェクトを検索する
~~~python
Book.objects.filter(price__gt=1000)
~~~
#### $<$を使った検索
`lt`（Less Than）を使う  
例：価格が1000円未満のBoolモデルのオブジェクトを検索する
~~~python
Book.objects.filter(price__lt=1000)
~~~

#### $>=$を使った検索
`gte`（Greater Than or Equal）を使う  
例：価格が1000円以上のBookモデルのオブジェクトを検索する
~~~python
Book.objects.filter(price__gte=1000)
~~~

#### $<=$を使った検索
`lte`（Less Than or Equal）を使う  
例：価格が1000円以下のBookモデルのオブジェクトを検索する
~~~python
Book.objects.filter(price__lte=1000)
~~~

#### IN句を使った検索
`in`を使う  
例：価格が900円または1000円のBookモデルのオブジェクトを検索する
~~~python
Book.objects.filter(price__in=[900, 1000])
~~~

#### LIKE句を使った検索
`icontains`または`contains`を使う  
`contains`は大文字小文字を区別し、`icontains`は大文字小文字を区別しない  
例：タイトルに「Django」（大文字小文字は区別しない）を含むBookモデルのオブジェクトを検索する  
~~~python
Book.objects.filter(title__icontains='Django')
~~~

### 6.7.6 リレーション先のモデルを使った条件検索
検索条件のキーワード引数として、以下のキーワードで指定する  
`<モデルのフィールド名>\_\_<リレーション先モデルのフィールド名>`  
例：publisher（出版社）の名称を使ってBookモデルのオブジェクトを検索する
~~~python
Book.objects.filter(publisher__name='自費出版社')
~~~

### 6.7.7 クエリセットオブジェクトの遅延評価のタイミング
クエリが発行されるタイミングは以下のように決まっている
- `for`ループなどのイテレーションが開始されたタイミング
- `[]`を使ってスライスしたタイミング
- オブジェクトを直列化したタイミング
- オブジェクトをREPL（対話型評価環境）や`print`で出力したタイミング
- `len()`でサイズを取得したタイミング
- `list()`で強制的にリストに変換したタイミング
- `bool()`で強制的にBooleanに変換したタイミング

#### 遅延評価に伴う弊害
モデルのリレーションに気をつけないと、想定外にクエリの数が増えてしまう可能性がある  
解決策については、[ベストプラクティス 6.11](#6.11-ベストプラクティス5：select_related/prefetch_relatedでクエリ本数を減らす)を参照

### 6.7.8 その他クエリセットAPIメソッド
### `exists`
レコードが存在するかどうかを`True`/`False`で返す  
例：Bookモデルに1つ以上のオブジェクトが存在するか、また、タイトルに「Flask」を含むBookモデルのオブジェクトが存在するか  
~~~python
# Bookモデルに1つ以上のオブジェクトが存在するか
Book.objects.all().exists()

# タイトルに「Flask」を含むBookモデルのオブジェクトが存在するか
Book.objects.filter(title__icontains='Flask').exists()
~~~

### `count`
レコードの件数を返す  
例：Bookモデルの全てのオブジェクトの件数と、タイトルに「Django」を含むBookモデルのオブジェクトの件数を取得する
~~~python
# Bookモデルの全てのオブジェクトの件数を取得
Book.objects.all().count()

# タイトルに「Django」を含むBookモデルのオブジェクトの件数を取得
Book.objects.filter(title__icontains='Django').count()
~~~

### `order_by`
検索結果をソートする  
デフォルトは昇順、降順でソートする場合は、指定するフィールド名の前に`-`を付ける  
複数フィールドでソートする場合には、カンマ区切りで列挙する  
例：Bookモデルの全てのオブジェクトをソートして取得する
~~~python
# 価格の昇順でソート
Book.objects.all().order_by('price')

# 価格の降順でソート
Book.objects.all().order_by('-price')

# 価格と出版日の昇順でソート
Book.objects.all().order_by('price', 'publish_date')
~~~

## 6.8 単体のオブジェクトを保存・更新・削除する
単体のオブジェクトを保存・更新するには`save()`メソッドを、削除するには`delete()`メソッドを使用する  
Djangoのデフォルト設定では、オブジェクトの保存・更新・削除は`save()`や`delete()`が実行された時点で即時にデータベースへ反映される（「オートコミットモード」）  

### 単体のオブジェクトを保存する
モデルクラスのコンストラクタにキーワード引数として値をセットするかインスタンスを作成してから属性に値をセットし、`save()`メソッドを実行する
#### `User`モデルのオブジェクトを作成して保存
~~~python
user = User(username='admin', is_active=True)
user.email = 'admin@example.com'
user.save()
~~~ 

### 単体のオブジェクトを更新する
データベースから取得したオブジェクトの属性値を変更して`save()`メソッドを実行する
#### `User`モデルのオブジェクトをデータベースから取得し、変更して保存
~~~python
user = User.objects.get(username='admin')
user.username = 'root'
user.save()
~~~

### 単体のオブジェクトを削除する
`delete()`メソッドを実行する
#### データベースから削除する
~~~python
user = User.objects.get(username='root')
user.delete()
~~~

### トランザクションの範囲を指定する
`with`ブロックで`django.db.transaction.atomic()`を使用する
#### `User`モデルの更新時にトランザクションを使用する
~~~python
from django.db import transaction

with transaction.atomic():
    User(username='aki').save()
    USer(username='aki').save()
~~~

### トランザクションのデフォルト設定を変更する
設定ファイルの`DATABESES.default`に`ATOMIC_REQUESTS`の記述を追加する
#### settings.py
~~~python
DATABASES = [
    'default': {
        ...
        'ATOMIC_REQUESTS' : True,
    }
]
~~~

## 6.9 ベストプラクティス3：Userモデルを拡張する
組み込みのUserモデルの拡張については、以下の方法が使用される
1. 抽象クラス`AbstractBaseUser`を継承する
1. 抽象クラス`AbstractUser`を継承する
1. 別モデルを作って`OneToOneField`で関連させる

まだ本番リリースしていない開発段階であれば、1.や2.の方法が推奨  
ただし、1.や2.の方法を使用する場合は、設定ファイルに設定の追加が必須となる  
すでに本番稼働している場合は、1.や2.の方法は関連しているモデルのスキーマが変更され、影響範囲が大きいため、3.の方法が推奨
### 抽象クラスを継承した拡張
任意のアプリケーションに`AbstractBaseUser`または`AbstractUser`を継承したカスタムユーザークラスを作成する  
どちらのクラスを継承するかはケースバイケースだが、ガラッと変えたいときは`AbstractBaseUser`を、Userモデルに少しだけ手を加えたいなら`AbstractUser`を選べば良い  
#### AbstractUserを継承したクラスの例
~~~python
from django.contrib.auth.models import AbstractUser
from django.db import models

class CustomUser(AbstractUser):
    """拡張ユーザーモデル"""
    class Meta:
        db_table = 'custom_user'
        
    login_count = models.IntegerField(verbose_name='ログイン回数', default=0)
~~~

#### 設定ファイルの追加（`config/settings.py`）
~~~python
AUTH_USER_MODEL = 'accounts.CustomUser'
~~~

#### 拡張Userクラスを使う
`django.contrib.auth.get_user_model()`関数を使って、拡張したUserモデルのクラスを取得する
~~~python
from django.contrib.auth import get_user_model
get_user_model()

# 拡張Userクラスの全てのオブジェクトを取得
get_user_model().objects.all()
~~~

### 別モデルを`OneToOneField`で連携させる拡張
#### edX（edX-platform）で利用されているUserモデルの拡張の例
~~~python
class UserProfile(models.Model):
    ...(略)...
    user = models.OneToOneField(User, unique=True. db_index=True, related_name='profile', on_delete=models.CASCADE)
    ...(略)...
    year_of_birth = models.IntegerField(blank=True, null=True, db_index=True)
    GENDER_CHOICES = (
        ('m', ugettext_noop('Male')),
        ('f', ugettext_noop('Female')),
        # Translators: 'Other' refers to the student's gender
        ('o', ugettext_noop('Other/Prefer Not to Say'))
    )
    gernder = models.CharField(blank=True, null=True, mac_length=6, db_index=True, choices=GENDER_CHOICES)
    ...(略)...
~~~

## 6.10 ベストプラクティス10：発行されるクエリを確認する

### （その１）Djangoシェルを使う
クエリセットオブジェクトの`query`属性を`print`することで、発行されるクエリを出力することが出来る
#### `query`属性を使ったクエリの確認
~~~python
print(Book.objects.filter(title__icontraints='Django').query)
~~~

### （その2）ロギングの設定を変更する
設定ファイルのロギング設定（`LOGGING`）をうまく設定することで、発行されるクエリをコンソールやログにリアルタイムで出力することが出来る  
詳細は第10章の「[10.7 ロギングの設定](第10章 設定オブジェクトと設定ファイル(settings.py).ipynb#10.7-ロギングの設定)」を参照


### （その3）django-debug-toolbarのSQLパネルを使う
`dk\jango-debug-toolbar`パッケージをインストールし、SQLパネルを使う  
一番手軽に発行されるクエリの内容を確認することが出来る  
実際に画面を操作しながら、SQLパネルを開いてクエリの内容を確認することが出来る  
設定方法については第14章の「[14.3.1 django-debug-toolbar（GUIによるデバッグ）](第14章 便利なDjangoパッケージを使おう.ipynb#14.3.1-django-debug-toolbar（GUIによるデバッグ）)」を参照

## 6.11 ベストプラクティス5：select_related/prefetch_relatedでクエリ本数を減らす
Bookモデルに対応するレコードの数が10の場合、次のような場合に発行されるクエリの数は11本となる（N+1問題）
~~~python
from book in Book.objects.all():
    print(book.publisher.name)
~~~

1. イテレーションが開始された時点でクエリが1本発行される
2. 10回ループする中で、リレーション先のオブジェクトを取得しようとしてそれぞれ1回ずつクエリが発行される

### `select_related`
リレーション先のオブジェクトを取得するために`JOIN`を使ったクエリを発行することが出来る
「一」や「多」側から「一」のリレーションのモデルオブジェクトを`JOIN`で取得する場合に使用する
#### `select_related`を使用したクエリの例
~~~python
Book.objects.all().select_related('publisher')
~~~
上記のコードを実行すると、以下の様のクエリが発行される
~~~sql
SELECT * FROM boook INNER JOIN publisher ON book.publisher_id = publisher.id
~~~

### `prefetch_related`
「一」や「多」側から「多」のリレーションのモデルを参照する場合に使用する  
取得したオブジェクト軍をオブジェクト内部のキャッシュに保持し、それを使い回すことで同じクエリが何回も発行されないようにしている  

#### `prefetch_related`を使用したクエリの例
~~~python
Book.objects.all().prefetch_related('authors')
~~~

## 6.12 まとめ
- モデルは`models.py`に書く
- モデルクラスは`django.db.models.Model`クラスを継承して作る
- モデルのフィールドには用途に合わせた`django.db.models.fields.Field`のサブクラスを定義する
- テーブルレベルのクエリ操作はモデルマネージャを経由してクエリセットAPIのメソッドを利用する
  - オブジェクトを1件取得する場合には`get()`を使う
  - 複数件取得する場合には`all()`または`filter()`を使う
- 行レベルのクエリ操作はモデルのメソッドを利用する
  - オブジェクトの保存・更新には`save()`を使う
  - オブジェクトの削除には`delete()`を使う
- クエリの発行回数がボトルネックにならないよう注意し、必要に応じて`select_related`や`prefetch_related`を使う